## End to end Deep Learning Project Using Simple RNN

In [2]:
import tensorflow as tf 
from tensorflow.keras.datasets import imdb 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense 
import numpy as np

In [4]:
vocab_size = 10000 # Keep top 10,000 most frequent words 
maxlen = 200 # Cut or pad reviews to 200 words

In [6]:
print("Loading dataset...") 
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")

Loading dataset...
Training samples: 25000, Test samples: 25000


In [7]:
X_train = pad_sequences(X_train, maxlen=maxlen) 
X_test = pad_sequences(X_test, maxlen=maxlen)

In [9]:
model = Sequential() 
model.add(Embedding(vocab_size, 128, input_length=maxlen)) # word embeddings 
model.add(SimpleRNN(64, activation='tanh')) # RNN layer 
model.add(Dense(1, activation='sigmoid')) # Binary classification 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
history = model.fit( X_train, y_train, epochs=5, batch_size=64, validation_split=0.2, verbose=1 )

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.6646 - loss: 0.5945 - val_accuracy: 0.7212 - val_loss: 0.5456
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.8436 - loss: 0.3593 - val_accuracy: 0.6814 - val_loss: 0.6038
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.9220 - loss: 0.2015 - val_accuracy: 0.8082 - val_loss: 0.4951
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.9787 - loss: 0.0696 - val_accuracy: 0.7812 - val_loss: 0.6411
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.9948 - loss: 0.0220 - val_accuracy: 0.7996 - val_loss: 0.7084


In [13]:
loss, acc = model.evaluate(X_test, y_test, verbose=0) 
print(f"\n✅ Test Accuracy: {acc*100:.2f}%")


✅ Test Accuracy: 79.94%


In [12]:
word_index = imdb.get_word_index()

In [15]:
reverse_word_index = {value+3: key for (key, value) in word_index.items()} 
reverse_word_index[0] = "<PAD>" 
reverse_word_index[1] = "<START>" 
reverse_word_index[2] = "<UNK>" 
reverse_word_index[3] = "<UNUSED>"

def review_to_sequence(review, word_index, maxlen=200): 
    """Convert raw text review into padded sequence of word IDs.""" 
    words = review.lower().split() 
    seq = [] 
    for word in words: 
        if word in word_index and word_index[word] < vocab_size: 
            seq.append(word_index[word]) 
        else: seq.append(2) # <UNK> for unknown words 
    return pad_sequences([seq], maxlen=maxlen)

In [23]:
custom_review = "The movie was worst that i have seen this year" 
seq_input = review_to_sequence(custom_review, word_index, maxlen=maxlen)

prediction = model.predict(seq_input)[0][0] 
sentiment = "Positive 😀" if prediction > 0.5 else "Negative 😞" 
print(f"\nReview: {custom_review}") 
print(f"Prediction: {prediction:.4f} → {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

Review: The movie was worst that i have seen this year
Prediction: 0.9906 → Positive 😀
